# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, see [this forum post](https://carnd-forums.udacity.com/cq/viewquestion.action?spaceKey=CAR&id=29496372&questionTitle=finding-lanes---import-cv2-fails-even-though-python-in-the-terminal-window-has-no-problem-with-import-cv2) for more troubleshooting tips.**  

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

/Users/bogdan/Workspaces/Udacity/anaconda3/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/bogdan/Workspaces/Udacity/anaconda3/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [450]:
def find_road_lanes(image, apex_x1_coord, apex_x2_coord, apex_y_coord):    
    gray = grayscale(image)
    gaussian = gaussian_blur(gray, 3)
    edges = canny(gaussian, 50, 150)

    vertices = np.array([[(0, image.shape[0]),(apex_x1_coord, apex_y_coord), (apex_x2_coord, apex_y_coord), (image.shape[1],image.shape[0])]], dtype=np.int32)
    masked_image = region_of_interest(edges, vertices)

    rho = 2              # distance resolution in pixels of the Hough grid
    theta = np.pi / 180  # angular resolution in radians of the Hough grid
    threshold = 15       # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 30 # minimum number of pixels making up a line
    max_line_gap = 10    # maximum gap in pixels between connectable line segments

    hough = hough_lines(masked_image, rho, theta, threshold, min_line_length, max_line_gap)
    final_image = weighted_img(image, hough)

    return final_image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [451]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    #return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    
    left_lane_x_coords = []
    left_lane_y_coords = []
    
    right_lane_x_coords = []
    right_lane_y_coords = []
    
    left_right_lane_x_coord_threshold = int(img.shape[1] / 2)
    
    # Carry out filtering for left and right lane
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1 > left_right_lane_x_coord_threshold and x2 > left_right_lane_x_coord_threshold:
                right_lane_x_coords.append(x1)
                right_lane_x_coords.append(x2)
                right_lane_y_coords.append(y1)
                right_lane_y_coords.append(y2)
            else: 
                left_lane_x_coords.append(x1)
                left_lane_x_coords.append(x2)
                left_lane_y_coords.append(y1)
                left_lane_y_coords.append(y2)
    
    if not left_lane_x_coords:
        return

    if not left_lane_y_coords:
        return
    
    if not right_lane_x_coords:
        return

    if not right_lane_y_coords:
        return
        
    # Find related y = ax + b form for each lane
    left_lane = np.polyfit(left_lane_x_coords, left_lane_y_coords, 1)
    right_lane = np.polyfit(right_lane_x_coords, right_lane_y_coords, 1)
    
    # Find bottom x coord for each lane
    left_lane_bottom_x = int((img.shape[0] - left_lane[1]) / left_lane[0])
    right_lane_bottom_x = int((img.shape[0] - right_lane[1]) / right_lane[0])

    # Define approx. function for each lane
    left_lane_function = np.poly1d(left_lane)
    right_lane_function = np.poly1d(right_lane)
    
    max_left_lane_x_coord = max(left_lane_x_coords)
    
    if max_left_lane_x_coord > left_right_lane_x_coord_threshold:
        max_left_lane_x_coord = left_right_lane_x_coord_threshold
    
    # Plot/draw function based on min and max x coord value for left lane
    for i in np.arange(left_lane_bottom_x, max_left_lane_x_coord - 10, 10):
        cv2.line(img,(i, int(left_lane_function(i))),(i + 10, int(left_lane_function(i + 10))), (255,0,0), 10)
    
    min_right_lane_x_coord = min(right_lane_x_coords)
    
    if min_right_lane_x_coord < left_right_lane_x_coord_threshold:
        min_right_lane_x_coord = left_right_lane_x_coord_threshold
        
    # Plot/draw function based on min and max x coord value for right lane
    for i in np.arange(min_right_lane_x_coord, right_lane_bottom_x - 10, 10):
        cv2.line(img,(i, int(right_lane_function(i))),(i + 10, int(right_lane_function(i + 10))), (255,0,0), 10)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
            
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [452]:
import os
import matplotlib.image as mpimg

test_images = os.listdir("test_images/")

run your solution on all test_images and make copies into the test_images directory).

In [453]:
test_images_folder = "test_images/"

for test_image in test_images:
    
    # Skip hidden files on Mac OSX
    if test_image == ".DS_Store":
        continue 
        
    image = mpimg.imread(test_images_folder + test_image)
    
    processed_image = find_road_lanes(image, 470, 490, 315)
    mpimg.imsave(test_images_folder + "processed_" + test_image, processed_image)
    
    continue



## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an `import error` when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt. Also, check out [this forum post](https://carnd-forums.udacity.com/questions/22677062/answers/22677109) for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://carnd-forums.udacity.com/display/CAR/questions/26218840/import-videofileclip-error) for more troubleshooting tips across operating systems.**

In [454]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [455]:
def process_image(image):
    result = find_road_lanes(image, 470, 490, 315)
    return result

Let's try the one with the solid white lane on the right first ...

In [456]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|█████████▉| 221/222 [00:02<00:00, 81.67it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 2.45 s, sys: 580 ms, total: 3.03 s
Wall time: 3.07 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [457]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [458]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|█████████▉| 681/682 [00:09<00:00, 72.65it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

CPU times: user 8.26 s, sys: 1.82 s, total: 10.1 s
Wall time: 9.83 s


In [234]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

The first project was awesome! My biggest challange was brushing up my Python skills and understanding which functionality I can use in OpenCV library. I've spent most of my time on the algorithm for lane extrapolations. I have to say that it took me a while to actually understand what was required from me. It might be due to the format of this Notebook. Maybe the detailed description of the task should be in the course material as well?

One idea for making the algorithm better would be better parameterization of my find_road_lanes method (in terms of area of interest). This is something which needs to be fixed in order for final challenge video to work properly. 


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [412]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|██████████| 251/251 [00:10<00:00, 24.06it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

CPU times: user 27.6 s, sys: 4.4 s, total: 32 s
Wall time: 11.4 s


In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))